#### 라이브러리

In [62]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import accuracy_score
import numpy as np
from xgboost import XGBClassifier
from tqdm import tqdm
import re

In [86]:
cur_df = pd.read_csv('curation_data/큐레이션_멜론_필터링_통합_컬러분할_이미지링크제거.csv')
cur_df.head(3)

,curColor,curPer,curColor2,curPer2,curColor3,curPer3,curObject,curHead,curTitle,curExplain,curArtist
0,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Like I Do,NaN,박재범
1,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Rewind,NaN,VVON
2,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Westcoast Marriage,NaN,Austn


In [87]:
melon_df = pd.read_csv('melon_data/멜론통합.csv')
melon_df.head(3)

,albumCov,title,artistIdx,albumIdx,genreIdx,releasedAt,lyrics
0,https://cdnimg.melon.co.kr/cm2/album/images/10...,Until I Found You,Stephen Sanchez,Until I Found You,포크,2021.09.01,"Georgia, Wrap me up in all your...\nI want ya'..."
1,https://cdnimg.melon.co.kr/cm2/album/images/11...,IT'S YOU (feat. keshi),MAX,LOVE IN STEREO,POP,2024.02.16,"Mm-hmm, no-oh, yeah, yeah\n\nIt's you\nLoving'..."
2,https://cdnimg.melon.co.kr/cm2/album/images/10...,UNDERSTAND,keshi,GABRIEL (Korean Special Edition),POP,2022.03.25,Never thought that I'd find\nThat the one in m...


In [88]:
cur_df.head(2)

,curColor,curPer,curColor2,curPer2,curColor3,curPer3,curObject,curHead,curTitle,curExplain,curArtist
0,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Like I Do,NaN,박재범
1,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Rewind,NaN,VVON


In [89]:
cur_df.head(2)

,curColor,curPer,curColor2,curPer2,curColor3,curPer3,curObject,curHead,curTitle,curExplain,curArtist
0,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Like I Do,NaN,박재범
1,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Rewind,NaN,VVON


In [90]:
# 괄호와 괄호 안의 내용을 제거하는 함수
def remove_parentheses_content(s):
    if pd.isna(s):
        return ''
    return re.sub(r'\(.*?\)', '', s).strip()

# 'artistIdx'와 'title' 컬럼에서 괄호와 그 안의 내용 제거
melon_df['artistIdx'] = melon_df['artistIdx'].apply(remove_parentheses_content)
melon_df['title'] = melon_df['title'].apply(remove_parentheses_content)

# 'curArtist'와 'curTitle' 컬럼에서 괄호와 그 안의 내용 제거
cur_df['curArtist'] = cur_df['curArtist'].apply(remove_parentheses_content)
cur_df['curTitle'] = cur_df['curTitle'].apply(remove_parentheses_content)

# 'song' 컬럼 생성: 아티스트와 제목을 결합
melon_df['song'] = melon_df['artistIdx'].astype(str) + ' - ' + melon_df['title']
cur_df['song'] = cur_df['curArtist'].astype(str) + ' - ' + cur_df['curTitle']

In [91]:
melon_df.head(2)

,albumCov,title,artistIdx,albumIdx,genreIdx,releasedAt,lyrics,song
0,https://cdnimg.melon.co.kr/cm2/album/images/10...,Until I Found You,Stephen Sanchez,Until I Found You,포크,2021.09.01,"Georgia, Wrap me up in all your...\nI want ya'...",Stephen Sanchez - Until I Found You
1,https://cdnimg.melon.co.kr/cm2/album/images/11...,IT'S YOU,MAX,LOVE IN STEREO,POP,2024.02.16,"Mm-hmm, no-oh, yeah, yeah\n\nIt's you\nLoving'...",MAX - IT'S YOU


In [92]:
cur_df.head(2)

,curColor,curPer,curColor2,curPer2,curColor3,curPer3,curObject,curHead,curTitle,curExplain,curArtist,song
0,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Like I Do,NaN,박재범,박재범 - Like I Do
1,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Rewind,NaN,VVON,VVON - Rewind


In [ ]:
for i, song in tqdm(enumerate(cur_df['song']), total=len(cur_df), desc='Processing cur_df'):
    # 2. melon_df에서 일치하는 행 찾기
    if song in melon_df['song'].values:
        # 3. 일치하는 행의 genre 값 추가
        cur_df.at[i, 'genre'] = melon_df.loc[melon_df['song'] == song, 'genreIdx'].values[0]

Processing cur_df: 100%|██████████| 6555/6555 [00:10<00:00, 635.82it/s]


In [98]:
cur_df

,curColor,curPer,curColor2,curPer2,curColor3,curPer3,curObject,curHead,curTitle,curExplain,curArtist,song,genre
0,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Like I Do,NaN,박재범,박재범 - Like I Do,R&B/Soul
1,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Rewind,NaN,VVON,VVON - Rewind,R&B/Soul
2,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Westcoast Marriage,NaN,Austn,Austn - Westcoast Marriage,R&B/Soul
3,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Unlock,NaN,차다빈,차다빈 - Unlock,R&B/Soul
4,Black,50.38,Grey,22.21,Grey,9.42,"['바지', '시력 관리', '고글', '색안경', '검정색과 흰색', '플래시 사...",드라이브할맛나게하는트렌디하고세련된알앤비국내힙합ㅣR&BHipHop,Beautiful,NaN,basecamp,basecamp - Beautiful,R&B/Soul
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6550,Grey,25.70,Pink,19.31,Grey,17.65,"['소매', '몸짓', '폰트', '하늘', '행복하다', '풍경', '상표', '...",선선한퇴근길에함께하고픈일렉사운드ㅣchillin' inthecaronmywayhome,Too Much To H8,#팝송 #플레이리스트 #드라이브,Paige,Paige - Too Much To H8,POP
6551,Grey,25.70,Pink,19.31,Grey,17.65,"['소매', '몸짓', '폰트', '하늘', '행복하다', '풍경', '상표', '...",선선한퇴근길에함께하고픈일렉사운드ㅣchillin' inthecaronmywayhome,War With Heaven,#팝송 #플레이리스트 #드라이브,keshi,keshi - War With Heaven,"R&B/Soul, 국외영화"
6552,Grey,25.70,Pink,19.31,Grey,17.65,"['하늘', '물', '낮', '식물', '세계', '빛', '자연', '나무', ...",영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,Been There Done That,#신나는 #플레이리스트 #여름,Notd,Notd - Been There Done That,POP
6553,Grey,25.70,Pink,19.31,Grey,17.65,"['하늘', '물', '낮', '식물', '세계', '빛', '자연', '나무', ...",영혼까지신나! 청량터지는트로피컬EDM플레이리스트 ️,Told You So,#신나는 #플레이리스트 #여름,HRVY,HRVY - Told You So,POP


In [95]:
cur_df.to_csv('machine_learing_data/머신러닝_통합.csv', index=False,encoding='utf-8-sig')